In [5]:
import h5py
import numpy as np
import pandas as pd
import itertools
import os

In [2]:
data_path = 'F:/Projects/PAE_PlaceCell/ProcessedData/'
spike_path = 'F:/Projects/PAE_PlaceCell/analysis/spikes/'
swr_df = pd.read_csv('F:/Projects/PAE_PlaceCell/analysis/swr_data/post_processed/swr_df.csv')


In [3]:
def add_group_affiliation(df):
    control=('RH13','RH14','LS21','LS23','LE2821','LE2823','LEM3116','LEM3120','LEM3216')
    pae=('RH11','RH16','LS17','LS19','LE2813','LEM3124','LEM3206','LEM3246')

    df['group'] = np.ones_like(df.session)

    regstr = '|'.join(control)
    idx = df['session'].str.upper().str.contains(regstr)    
    df.group[idx] = 'control'

    regstr = '|'.join(pae)
    idx = df['session'].str.upper().str.contains(regstr)    
    df.group[idx] = 'pae'
    return df

def add_epoch_type(df,data_path):
    dicts = {}
    for session in df.session:
        f = h5py.File(data_path+session+'.mat','r')
        ex_ep = []
        for i in range(f['events'].shape[0]):
            ex_ep.append(f['events'][i])
        dicts[session] = ex_ep

    ep_type = ['pedestal_1','track','pedestal_2','cylinder_1','pedestal_3','cylinder_2','pedestal_4']
    df['ep_type'] = np.ones_like(df.session)
    # session_df=pd.DataFrame()
    for session in np.unique(df.session):
        # stack epoch times
        b = np.hstack(dicts[session])

        # add 0 to start to indicate the start of the recording session
        b = np.insert(b,0,0)

        # add the ts of the last ripple of the session to indicate end of session
        b = list(b)
        last_rip = max(df.ts[df.session == session])
        if b[-1] < last_rip:
            b.append(last_rip)

        # loop through each epoch and label each ripple
        for ep in range(len(b)-1):
            idx = (df.session == session) & (df.ts >= b[ep]) & (df.ts <= b[ep+1])
            df.loc[idx,'ep_type'] = ep_type[ep]

    return df

In [ ]:
ep_types = []
epoch_durations = []
sessions = []
for session in pd.unique(swr_df.session):
    print(session)
    # load spikes to get max and min of session
    spikes = np.load(os.path.join(spike_path,session)+'.npy', allow_pickle=True)
    spikes_ = list(itertools.chain(*spikes))
    # generate time stamps with dt of .5 seconds
    ts = np.arange(min(spikes_), max(spikes_),.5)
    # make df to feed into add_epoch_type
    df_test = pd.DataFrame()
    df_test['session'] = np.full(ts.shape[0], session)
    df_test['ts'] = ts
    df_test = add_epoch_type(df_test,data_path)
    # loop through each ep_type to collect ep type and epoch durations
    for ep_type in pd.unique(df_test.ep_type):
        sessions.append(session)
        ep_types.append(ep_type)
        epoch_durations.append(df_test.ts[df_test.ep_type == ep_type].max() - df_test.ts[df_test.ep_type == ep_type].min())


In [ ]:
epoch_df = pd.DataFrame()

epoch_df['session'] = sessions
epoch_df['ep_type'] = ep_types
epoch_df['epoch_duration'] = epoch_durations

epoch_df = add_group_affiliation(epoch_df)    
epoch_df

epoch_df.to_csv('F:/Projects/PAE_PlaceCell/analysis/epoch_df.csv')
